<a href="https://colab.research.google.com/github/TReV-89/TReV-89/blob/main/LanguuageID.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

LANGUAGE MODEL USING NAIVE BAYES ALGORITHM N-GRAMS

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB, ComplementNB, BernoulliNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report,accuracy_score, confusion_matrix

In [2]:
df = pd.read_json("/content/drive/MyDrive/salt-train-v1.2.jsonl", lines = True)
df.pop("tts-speech")
data = {
    "language": [],
    "text": []
}
for row in df['text']:
    for key, value in row.items():
        data["language"].append(key)
        data["text"].append(value)
df1 = pd.DataFrame(data,columns = ['language','text'])
df1.head()


,language,text
0,eng,It was not a ghost refugee camp.
1,lug,Enkambi y'abanoonyiboobubudamu teyaliiwo mu bu...
2,ach,Pe obedo kem goba goba
3,teo,Mam arai ekabi lo erai ekwam.
4,lgg,Eri aa'ni ndra kembe emunyale eyini aa'zu inzo...


In [3]:
X = df1['text']
Y = df1['language']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33, random_state=42)


DEPLOY COUNTVECTORIZER AND PIPELINE FUNCTIONS TO BUILD AND TRAIN MODELS FOR MULTINOMIAL_NB, BERNOULLI_NB & COMPLEMENT_NB MODELS

In [49]:
count = CountVectorizer(analyzer = 'char',ngram_range=(2,4))

pipeline_MultinomialNB = Pipeline([
   ('vectorizer',count),
   ('model',MultinomialNB())
])
pipeline_MultinomialNB.fit(X_train,Y_train)
Y_pred = pipeline_MultinomialNB.predict(X_test)

print(classification_report(Y_test,Y_pred))

accuracy = accuracy_score(Y_test, Y_pred)
print("Accuracy:", accuracy)

              precision    recall  f1-score   support

         ach       1.00      1.00      1.00      7826
         eng       1.00      1.00      1.00      7783
         lgg       1.00      1.00      1.00      8089
         lug       1.00      1.00      1.00      7799
         nyn       1.00      1.00      1.00      7967
         teo       1.00      1.00      1.00      7952

    accuracy                           1.00     47416
   macro avg       1.00      1.00      1.00     47416
weighted avg       1.00      1.00      1.00     47416

Accuracy: 0.9988822338451155


ACCURACY FOR MULTINOMIAL_NB IS 99.888% USING NGRAM_RANGE OF 2-4

In [52]:
count = CountVectorizer(analyzer = 'char',ngram_range=(2,4))

pipeline_BernoulliNB = Pipeline([
   ('vectorizer',count),
   ('model',BernoulliNB())
])
pipeline_BernoulliNB.fit(X_train,Y_train)
Y_pred = pipeline_BernoulliNB.predict(X_test)

print(classification_report(Y_test,Y_pred))

accuracy = accuracy_score(Y_test, Y_pred)
print("Accuracy:", accuracy)

              precision    recall  f1-score   support

         ach       1.00      1.00      1.00      7826
         eng       1.00      1.00      1.00      7783
         lgg       1.00      1.00      1.00      8089
         lug       1.00      1.00      1.00      7799
         nyn       1.00      1.00      1.00      7967
         teo       1.00      1.00      1.00      7952

    accuracy                           1.00     47416
   macro avg       1.00      1.00      1.00     47416
weighted avg       1.00      1.00      1.00     47416

Accuracy: 0.9988611439176649


ACCURACY FOR BERNOULLI_NB IS 99.886% USING NGRAM_RANGE OF 2-4

In [50]:
count = CountVectorizer(analyzer = 'char',ngram_range=(2,4))

pipeline_ComplementNB = Pipeline([
   ('vectorizer',count),
   ('model',ComplementNB())
])
pipeline_ComplementNB.fit(X_train,Y_train)
Y_pred = pipeline_ComplementNB.predict(X_test)

print(classification_report(Y_test,Y_pred))

accuracy = accuracy_score(Y_test, Y_pred)
print("Accuracy:", accuracy)

              precision    recall  f1-score   support

         ach       1.00      1.00      1.00      7826
         eng       0.99      1.00      1.00      7783
         lgg       1.00      1.00      1.00      8089
         lug       1.00      1.00      1.00      7799
         nyn       1.00      1.00      1.00      7967
         teo       1.00      1.00      1.00      7952

    accuracy                           1.00     47416
   macro avg       1.00      1.00      1.00     47416
weighted avg       1.00      1.00      1.00     47416

Accuracy: 0.9977222878353299


ACCURACY FOR COMPLEMENT_NB IS 99.772% USING NGRAM_RANGE OF 2-4

In [65]:
import joblib

joblib.dump(pipeline_BernoulliNB ,'pipeline_BernoulliNB.pkl',compress = ('lzma',9))
pipeline_BNB = joblib.load('pipeline_BernoulliNB.pkl')
joblib.dump(pipeline_MultinomialNB ,'pipeline_MultinomialNB.pkl',compress = ('lzma',9))
pipeline_MNB = joblib.load('pipeline_MultinomialNB.pkl')
joblib.dump(pipeline_ComplementNB ,'pipeline_ComplementNB.pkl',compress = ('lzma',9))
pipeline_CNB = joblib.load('pipeline_ComplementNB.pkl')


TEST MODELS ON SALT TEST DATASET

In [16]:
df2 = pd.read_json("/content/drive/MyDrive/salt-test-v1.2.jsonl", lines = True)
df2.pop("tts-speech")
data1 = {
    "language": [],
    "text": []
}
for row in df2['text']:
    for key, value in row.items():
        data1["language"].append(key)
        data1["text"].append(value)
df3 = pd.DataFrame(data1,columns = ['language','text'])
df3.head()


,language,text
0,eng,The fashion industry is starting to thrive again.
1,lug,Ekisaawe ky'emisono kitandise okusituka nate.
2,ach,Yub me cital ruk mapatpat manyen tye ka dongo ...
3,teo,Ageutu ikampunin luka enape apolo bobo.
4,lgg,Okalamvu suta o'diru 'diyini 'diyi 'ye e'do tutu


In [55]:
X = df3['text']
Y = df3['language']

Y_pred = pipeline_MNB.predict(X)

print(classification_report(Y,Y_pred))

accuracy = accuracy_score(Y, Y_pred)
print("Accuracy:", accuracy)
print("\nConfusion Matrix:")
print(confusion_matrix(Y, Y_pred))

              precision    recall  f1-score   support

         ach       1.00      1.00      1.00       500
         eng       1.00      1.00      1.00       500
         lgg       1.00      1.00      1.00       500
         lug       1.00      1.00      1.00       500
         nyn       1.00      1.00      1.00       500
         teo       1.00      1.00      1.00       500

    accuracy                           1.00      3000
   macro avg       1.00      1.00      1.00      3000
weighted avg       1.00      1.00      1.00      3000

Accuracy: 0.9986666666666667

Confusion Matrix:
[[498   0   1   1   0   0]
 [  0 500   0   0   0   0]
 [  0   1 499   0   0   0]
 [  0   0   0 500   0   0]
 [  0   0   0   1 499   0]
 [  0   0   0   0   0 500]]


ACCURACY FOR MULTINOMIAL_NB IS 99.867%

In [56]:
Y_pred = pipeline_CNB.predict(X)

print(classification_report(Y,Y_pred))

accuracy = accuracy_score(Y, Y_pred)
print("Accuracy:", accuracy)
print("\nConfusion Matrix:")
print(confusion_matrix(Y, Y_pred))

              precision    recall  f1-score   support

         ach       1.00      1.00      1.00       500
         eng       0.99      1.00      1.00       500
         lgg       1.00      1.00      1.00       500
         lug       1.00      1.00      1.00       500
         nyn       1.00      1.00      1.00       500
         teo       1.00      1.00      1.00       500

    accuracy                           1.00      3000
   macro avg       1.00      1.00      1.00      3000
weighted avg       1.00      1.00      1.00      3000

Accuracy: 0.9976666666666667

Confusion Matrix:
[[499   0   1   0   0   0]
 [  0 500   0   0   0   0]
 [  0   2 498   0   0   0]
 [  0   1   0 498   1   0]
 [  0   0   0   2 498   0]
 [  0   0   0   0   0 500]]


ACCURACY FOR COMPLEMENT_NB IS 99.767%

In [57]:
Y_pred = pipeline_BNB.predict(X)

print(classification_report(Y,Y_pred))

accuracy = accuracy_score(Y, Y_pred)
print("Accuracy:", accuracy)
print("\nConfusion Matrix:")
print(confusion_matrix(Y, Y_pred))

              precision    recall  f1-score   support

         ach       1.00      1.00      1.00       500
         eng       1.00      1.00      1.00       500
         lgg       1.00      1.00      1.00       500
         lug       1.00      1.00      1.00       500
         nyn       1.00      1.00      1.00       500
         teo       1.00      1.00      1.00       500

    accuracy                           1.00      3000
   macro avg       1.00      1.00      1.00      3000
weighted avg       1.00      1.00      1.00      3000

Accuracy: 0.9986666666666667

Confusion Matrix:
[[499   0   1   0   0   0]
 [  0 500   0   0   0   0]
 [  0   1 499   0   0   0]
 [  0   0   0 500   0   0]
 [  0   0   0   2 498   0]
 [  0   0   0   0   0 500]]


ACCURACY FOR BERNOULLI_NB IS 99.867%

TESTED MODELS ON SAMPLES SENTENCES PROVIDED BY MR. ISAAC



In [25]:
new_samples = ['I am going to the katale', # eng
'Ngenda mu katale', # lug
'Uganda', # any
'Ekikopo', # lug or nyn
'Runyankole', # nyn or lug
'Omuntu', # nyn or lug
"Ebigambo by'amagezi", # lug
"Bw'okoowa", # lug
"yiga", # lug
"kubivaako", # lug
"Mulindeeko tujja kubayita", # lug
"Gamba Mpurire", # nyn
"Ekiro twaburayo kimwe kyonka tukajaguza embaga y'omwaka", # nyn
"Tushangye", # nyn
"Ekitongore ekya UWA kiriyo nikibangura abanyamakuru kuruga omukyanga ekya Kigezi ahakuhandiika amakuru agakwatiraine n’okurinda enyamaishwa z’omukishaka.", # nyn
"itye nining", # ach
"Irii maber, ladit?", # ach
"Ibuto maber", # ach
"Ibuto Gulu", # ach,
"Amito tedo labolo kwon.", # ach
"Ngo manyen?", # ach
"Wacito I gang kwan nino ducu", # ach
"Kumbedi abedo Anaka i Amuru aa ki California i Amerika ento." # ach
]

FOR THE MULTINOMIALNB MODEL

In [60]:
y_pred_new = pipeline_MNB.predict(new_samples)

for sentence, language in zip(new_samples, y_pred_new):
    print(f"Text: '{sentence}' => Predicted Language: {language}")

Text: 'I am going to the katale' => Predicted Language: eng
Text: 'Ngenda mu katale' => Predicted Language: lug
Text: 'Uganda' => Predicted Language: lug
Text: 'Ekikopo' => Predicted Language: teo
Text: 'Runyankole' => Predicted Language: lug
Text: 'Omuntu' => Predicted Language: nyn
Text: 'Ebigambo by'amagezi' => Predicted Language: lug
Text: 'Bw'okoowa' => Predicted Language: lug
Text: 'yiga' => Predicted Language: lug
Text: 'kubivaako' => Predicted Language: lug
Text: 'Mulindeeko tujja kubayita' => Predicted Language: lug
Text: 'Gamba Mpurire' => Predicted Language: nyn
Text: 'Ekiro twaburayo kimwe kyonka tukajaguza embaga y'omwaka' => Predicted Language: nyn
Text: 'Tushangye' => Predicted Language: nyn
Text: 'Ekitongore ekya UWA kiriyo nikibangura abanyamakuru kuruga omukyanga ekya Kigezi ahakuhandiika amakuru agakwatiraine n’okurinda enyamaishwa z’omukishaka.' => Predicted Language: nyn
Text: 'itye nining' => Predicted Language: ach
Text: 'Irii maber, ladit?' => Predicted Language

Failed to detect "ekikopo"

FOR THE BERNOUILLINB MODEL

In [61]:
y_pred_new = pipeline_BNB.predict(new_samples)

for sentence, language in zip(new_samples, y_pred_new):
    print(f"Text: '{sentence}' => Predicted Language: {language}")

Text: 'I am going to the katale' => Predicted Language: eng
Text: 'Ngenda mu katale' => Predicted Language: lug
Text: 'Uganda' => Predicted Language: ach
Text: 'Ekikopo' => Predicted Language: teo
Text: 'Runyankole' => Predicted Language: lug
Text: 'Omuntu' => Predicted Language: lug
Text: 'Ebigambo by'amagezi' => Predicted Language: lug
Text: 'Bw'okoowa' => Predicted Language: lug
Text: 'yiga' => Predicted Language: ach
Text: 'kubivaako' => Predicted Language: lug
Text: 'Mulindeeko tujja kubayita' => Predicted Language: lug
Text: 'Gamba Mpurire' => Predicted Language: nyn
Text: 'Ekiro twaburayo kimwe kyonka tukajaguza embaga y'omwaka' => Predicted Language: nyn
Text: 'Tushangye' => Predicted Language: nyn
Text: 'Ekitongore ekya UWA kiriyo nikibangura abanyamakuru kuruga omukyanga ekya Kigezi ahakuhandiika amakuru agakwatiraine n’okurinda enyamaishwa z’omukishaka.' => Predicted Language: nyn
Text: 'itye nining' => Predicted Language: ach
Text: 'Irii maber, ladit?' => Predicted Language

FAILED TO DETECTED "EKIKOPO" & "YIGA"

In [62]:
y_pred_new = pipeline_CNB.predict(new_samples)

for sentence, language in zip(new_samples, y_pred_new):
    print(f"Text: '{sentence}' => Predicted Language: {language}")

Text: 'I am going to the katale' => Predicted Language: eng
Text: 'Ngenda mu katale' => Predicted Language: lug
Text: 'Uganda' => Predicted Language: nyn
Text: 'Ekikopo' => Predicted Language: teo
Text: 'Runyankole' => Predicted Language: lug
Text: 'Omuntu' => Predicted Language: nyn
Text: 'Ebigambo by'amagezi' => Predicted Language: lug
Text: 'Bw'okoowa' => Predicted Language: lug
Text: 'yiga' => Predicted Language: lug
Text: 'kubivaako' => Predicted Language: lug
Text: 'Mulindeeko tujja kubayita' => Predicted Language: lug
Text: 'Gamba Mpurire' => Predicted Language: nyn
Text: 'Ekiro twaburayo kimwe kyonka tukajaguza embaga y'omwaka' => Predicted Language: nyn
Text: 'Tushangye' => Predicted Language: nyn
Text: 'Ekitongore ekya UWA kiriyo nikibangura abanyamakuru kuruga omukyanga ekya Kigezi ahakuhandiika amakuru agakwatiraine n’okurinda enyamaishwa z’omukishaka.' => Predicted Language: nyn
Text: 'itye nining' => Predicted Language: ach
Text: 'Irii maber, ladit?' => Predicted Language

Failed to detect "ekikopo"